# Eigenvalues

## Eigenvalue decomposition

When it exists, the eigenvalue decomposition (EVD) expresses a square matrix $A$ as a change of basis (in both source and image spaces) to a  diagonal matrix: $A=XDX^{-1}$. There are two major syntaxes. If you call `eigen`, you get the complete EVD. 

In [1]:
using LinearAlgebra
A = [
    16     2     3    13
     5    11    10     8
     9     7     6    12
     4    14    15     1

];
(λ,X) = eigen(A);
@show λ;
@show X;

λ = [34.0, 8.94427, -8.94427, 2.05764e-15]
X = [-0.5 -0.823607 0.376393 -0.223607; -0.5 0.423607 0.0236068 -0.67082; -0.5 0.0236068 0.423607 0.67082; -0.5 0.376393 -0.823607 0.223607]


Technically, the output is guaranteed only to satisfy $AX=XD$, which holds for defective as well as diagonalizable matrices. (But defectiveness is a "probability zero" event and is not robust to floating point perturbations.)

In [2]:
A = 5I + diagm(1=>[1;1;1])   # 4x4 Jordan block
(λ,X) = eigen(A);
@show λ
@show rank(X)

λ = [5.0, 5.0, 5.0, 5.0]
rank(X) = 1


1

In [3]:
D = diagm(0=>λ);
@show residual = norm(A*X-X*D);

residual = norm(A * X - X * D) = 1.538370149106851e-15


You can request the eigenvalues alone using `eigvals`. This can be significantly faster than the full EVD.

## Schur factorization

The EVD has some serious drawbacks as a target for numerical computation. First, it does not exist for every square matrix. And second, if it exists but the condition number of the eigenvalue matrix is large, it will be very sensitive to roundoff. A stable alternative is the **Schur factorization** $A=QTQ^*$, in which $Q$ is unitary and $T$ is upper triangular.

In [4]:
A = [
    17    24     1     8    15
    23     5     7    14    16
     4     6    13    20    22
    10    12    19    21     3
    11    18    25     2     9
];

(T,Q) = schur(A);
T

5×5 Array{Float64,2}:
 65.0    5.3192e-15   -2.099e-15   2.71171e-15   3.63509e-15
  0.0  -21.2768       -2.58883     2.18707      -3.48932    
  0.0    0.0         -13.1263     -3.38447      -2.82394    
  0.0    0.0           0.0        21.2768        2.62871    
  0.0    0.0           0.0         0.0          13.1263     

The Schur form always exists, it still reveals the eigenvalues of $A$, and it involves only stable unitary transformations. 

There is one quirk. If $A$ is real but has complex eigenvalues, there is an alternate form in which $T$ is "quasitriangular." Each real eigenvalue appears along the diagonal of $T$, but complex conjugate pairs are represented as $2\times 2$ real blocks along the diagonal. This allows the entire factorizaton to be done in real arithmetic, which can improve speed. 

In [5]:
A = rotr90(A);
λ = eigvals(A)

5-element Array{Complex{Float64},1}:
       65.00000000000003 + 0.0im               
  1.7763568394002505e-15 + 21.276765471473805im
  1.7763568394002505e-15 - 21.276765471473805im
 -1.7763568394002505e-15 + 13.126280930709225im
 -1.7763568394002505e-15 - 13.126280930709225im

In [6]:
(T,Q) = schur(A);
T

5×5 Array{Float64,2}:
 65.0  -2.93387e-15   -5.45074e-16    4.63146e-15  -3.27119e-15
  0.0   1.77636e-15  -21.5782        -5.59552e-14   1.70351    
  0.0  20.9795         1.77636e-15   -6.8026       -5.14033e-14
  0.0   0.0            0.0           -1.77636e-15  13.4714     
  0.0   0.0            0.0          -12.79         -1.77636e-15

In [7]:
eigvals(T[2:3,2:3])

2-element Array{Complex{Float64},1}:
 1.7763568394002505e-15 + 21.276765471473805im
 1.7763568394002505e-15 - 21.276765471473805im

## Hessenbergification

Eigenvalue algorithms are iterative. The iterative steps can be performed a lot faster if the matrix has structural zeros. Hence before iteration begins, an orthogonal similarity transformation is used to get an **upper Hessenberg** matrix with the same eigenvalues.

In [8]:
hess = hessenberg(A);
hess.H

5×5 Array{Float64,2}:
  11.0     -14.8962   26.3553    4.02245  -1.14879 
 -32.1559   35.5909  -15.0266   -6.44772   1.46891 
   0.0     -32.1341   15.4136   -4.83816  -1.38772 
   0.0       0.0      10.0227    2.74592  17.3583  
   0.0       0.0       0.0     -13.96      0.249603

In [9]:
eigvals(hess.H)

5-element Array{Complex{Float64},1}:
       65.00000000000003 + 0.0im               
  1.7763568394002505e-15 + 21.276765471473805im
  1.7763568394002505e-15 - 21.276765471473805im
 -1.7763568394002505e-15 + 13.126280930709225im
 -1.7763568394002505e-15 - 13.126280930709225im

Note that $A=QHQ^*$ is "nearly" a Schur factorization. All that remains to be done is to make the subdiagonal zero. 

If $A$ is hermitian, then since the transformation preserves symmetry the resulting $H$ is actually hermitian and tridiagonal. This leads to even greater speedup in eigenvalue computations; for this and other reasons, one might even consider the hermitian and nonhermitian problems to be of distinct species.

In [10]:
A = A+A';
hessenberg(A).H

5×5 Array{Float64,2}:
  22.0     -54.0925   -2.24993e-15  4.90454e-15  -9.19126e-15
 -54.0925  102.463    -7.58203      5.00738e-15  -1.00276e-14
   0.0      -7.58203   0.498749     1.52404      -1.9984e-15 
   0.0       0.0       1.52404      4.87467       3.28539    
   0.0       0.0       0.0          3.28539       0.163146   

In [11]:
Tridiagonal(ans)

5×5 Tridiagonal{Float64,Array{Float64,1}}:
  22.0     -54.0925     ⋅         ⋅        ⋅      
 -54.0925  102.463    -7.58203    ⋅        ⋅      
    ⋅       -7.58203   0.498749  1.52404   ⋅      
    ⋅         ⋅        1.52404   4.87467  3.28539 
    ⋅         ⋅         ⋅        3.28539  0.163146

## Outer product interpretation

We can view the diagonalization $A=XDX^{-1}$ as $A=XDY^*$, where $Y^*X=I$. The rows of $Y^*$ are known as **left eigenvectors** of $A$.

In [12]:
A = [
    16     2     3    13
     5    11    10     8
     9     7     6    12
     4    14    15     1

];
(λ,X) = eigen(A);
D = diagm(0 => λ);
Y = inv(X')

4×4 Array{Float64,2}:
 -0.5  -0.934017  -0.184017  -0.223607
 -0.5   0.684017   0.434017  -0.67082 
 -0.5   0.434017   0.684017   0.67082 
 -0.5  -0.184017  -0.934017   0.223607

In [13]:
( Y[:,1]'*A, λ[1]Y[:,1] )

([-17.0 -17.0 -17.0 -17.0], [-17.0, -17.0, -17.0, -17.0])

They are also the (right) eigenvectors of $A^*$. (If $A^*=A$, then of course $X=Y$ is orthogonal.)

In [14]:
( A'*Y[:,1], λ[1]Y[:,1] )

([-17.0, -17.0, -17.0, -17.0], [-17.0, -17.0, -17.0, -17.0])

Hence we can write $A=\sum \lambda_k x_k y_k^*$. While this is interesting, so far I don't know if it's very useful to us. 